<a href="https://colab.research.google.com/github/martskow/Engineering-thesis/blob/main/Dane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Marta Skowron
# Przygotowanie danych pod kątem pracy inżynierszkiej / Preparing data for engineering thesis
"Projektowanie peptydów amyloidogennych z wykorzystaniem sieci neuronowych"

"Design of amyloidogenic peptides using neural networks"

### Importy / Imports

In [ ]:
#!pip install biopython
from Bio.Seq import Seq
from Bio.SeqUtils.IsoelectricPoint import IsoelectricPoint as IEP
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqUtils import ProtParam
import pandas as pd
import numpy as np

import requests
from urllib.parse import quote
from bs4 import BeautifulSoup

Załadowanie odpowiednich kolumn z plików oraz przetworzenie ich / Loading the appropriate columns from files and processing them:


*   '/41598_2015_BFsrep12494_MOESM2_ESM.xls'
*   '/1-s2.0-S0022283620305805-mmc11.xlsx'



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_xls = '/content/drive/MyDrive/Colab Notebooks/41598_2015_BFsrep12494_MOESM2_ESM.xls'
xls_data = pd.read_excel(file_xls, engine='xlrd')

# Columns E, J, N with the characters ' ' and '-' removed
col_e = xls_data.iloc[:, 4].dropna().astype(str).str.replace('[- ]', '', regex=True).str.upper()
col_j = xls_data.iloc[:, 9].dropna().astype(str).str.replace('[- ]', '', regex=True).str.upper()
col_n = xls_data.iloc[:, 13].dropna().astype(str).str.replace('[- ]', '', regex=True).str.upper()

combined_column = pd.concat([col_e, col_j, col_n], ignore_index=True)

df_combined = pd.DataFrame({
    'sequence': combined_column
})

# Assigning an id to each row
df_combined['id'] = range(1, len(df_combined) + 1)

In [ ]:
file_xlsx = '/content/drive/MyDrive/Colab Notebooks/1-s2.0-S0022283620305805-mmc11.xlsx'
df = pd.read_excel(file_xlsx)

# Row selection (range 4-300 and 305-353)
df_selected = pd.concat([df.iloc[3:300], df.iloc[304:353]]).reset_index(drop=True)

# Columns I and J from XLSX file with id
col_c = df_selected.iloc[:, 2].dropna().astype(str)  # column C (ID dla I)
col_f = df_selected.iloc[:, 5].dropna().astype(str)  # column F (ID dla J_2)

col_i = df_selected.iloc[:, 8].dropna().astype(str).str.replace('[- ]', '', regex=True).str.upper()  # ccolumn I
col_j_2 = df_selected.iloc[:, 9].dropna().astype(str).str.replace('[- ]', '', regex=True).str.upper() # column J

# Checking if the length of the id column matches the length of the sequence column
if len(col_c) == len(col_i):
    df_i = pd.DataFrame({'id': col_c, 'sequence': col_i})
else:
    raise ValueError("Kolumny C i I mają różne długości!")

if len(col_f) == len(col_j_2):
    df_j_2 = pd.DataFrame({'id': col_f, 'sequence': col_j_2})
else:
    raise ValueError("Kolumny F i J_2 mają różne długości!")

In [ ]:
df_result = pd.concat([df_i, df_j_2, df_combined], ignore_index=True)
print(df_result)

              id                     sequence
0     CAB66307.1  RHVHLRARASGSARIYQAGRDQHITER
1     KKD13728.1  RHVHLRARASGSARIYQAGRDQHITER
2     PSK58951.1  RHVHLRARASGSARIYQAGRDQHITER
3     SDT45679.1  RHVHLRARASGSARIYQAGRDQHITER
4     ANJ07606.1  RDVHLRARASGSARIYQAGRDQHITER
...          ...                          ...
1091         404        VSNAVGDLRIEGHGRAHVGNI
1092         405        HTVTYTGNRAEGNSTNHYGHN
1093         406        TMLSVFLTFELERLFAVTGDQ
1094         407         TGHRYLGTTIHGRGHHLGDH
1095         408        HHQHTSGVSTQNHSQAFVGSA

[1096 rows x 2 columns]


### Obliczenie własności peptydów / Calculation of peptide properties

#### Punkt izoelektryczny / Isoelectric point

In [ ]:
def calculate_isoelectric_point(sequence):
    """
    Calculate the isoelectric point (pI) of a given peptide sequence.

    Args:
      sequence (str): A string representing the peptide sequence, consisting
                    of standard amino acid symbols.

    Returns:
      float: The calculated isoelectric point (pI) of the peptide.
    """
    seq = Seq(sequence)
    analyzer = IEP(seq)
    return analyzer.pi()

df_result['isoelectric_point'] = df_result['sequence'].apply(calculate_isoelectric_point)

#### Peptide charge vector for pH 4 to 9

In [ ]:
def calculate_charge_vector(sequence):
    """
    Calculates the charge vector of a protein sequence over a pH range from 4 to 9.

    Args:
        sequence (str): The amino acid sequence of the protein.

    Returns:
        list: A list containing the net charge of the sequence at pH values from 4 to 9.
    """
    seq = Seq(sequence)
    analyzer = IEP(seq)
    charge_vector = [analyzer.charge_at_pH(pH) for pH in range(4, 10)]  # pH from 4 to 9
    return charge_vector

df_result['charge_vector'] = df_result['sequence'].apply(calculate_charge_vector)

#### Podział sekwencji na 5 fragmentów / Splitting the sequence into 5 fragments

In [ ]:
def split_sequence(sequence):
    """
    Splits a given sequence into 5 fragments, ensuring an even distribution of characters.

    This function divides the input sequence into 5 fragments. If the sequence length is not
    perfectly divisible by 5, the N-terminus (first fragment) and C-terminus (last fragment)
    will contain one additional character until the remainder is distributed. The middle fragments
    will adjust accordingly to ensure an even split.

    Args:
        sequence (str): The input sequence (e.g., a protein or nucleotide sequence).

    Returns:
        list: A list of 5 fragments where each fragment contains a portion of the sequence.
    """
    length = len(sequence)
    fragment_size = length // 5
    remainder = length % 5

    fragments = [
        sequence[:fragment_size + (1 if remainder > 0 else 0)],  # N-terminus
        sequence[fragment_size + (1 if remainder > 0 else 0):2*fragment_size + (1 if remainder > 1 else 0)],  # Middle 1
        sequence[2*fragment_size + (1 if remainder > 1 else 0):3*fragment_size + (1 if remainder > 2 else 0)],  # Middle 2
        sequence[3*fragment_size + (1 if remainder > 2 else 0):4*fragment_size + (1 if remainder > 3 else 0)],  # Middle 3
        sequence[4*fragment_size + (1 if remainder > 3 else 0):]  # C-terminus
    ]

    return fragments

#### Macierz ładunków peptydu po jego podzieleniu na 5 fragmentów dla pH od 4 do 9 / Charge matrix of the peptide after its division into 5 fragments for pH 4 to 9

In [ ]:
def calculate_charge_matrix(sequence):
    """
    Calculates a charge matrix for a protein sequence by dividing it into 5 fragments and determining the charge
    of each fragment across a pH range from 4 to 9.

    This function splits the input sequence into 5 equal or nearly equal fragments using the split_sequence() function.
    For each fragment, it calculates the net charge at six different pH values (from pH 4 to 9). The result is a 5x6 matrix,
    where each row represents a fragment, and each column represents the charge at a specific pH value.

    Args:
        sequence (str): The amino acid sequence of the protein.

    Returns:
        list: A 2D list (matrix) where each sublist contains the net charges of one fragment at pH values 4 to 9.
              The matrix will have 5 rows (for the 5 fragments) and 6 columns (for the 6 pH values).
    """
    seq_fragments = split_sequence(sequence)  # Split the sequence into 5 fragments
    charge_matrix = []

    for fragment in seq_fragments:
        seq = Seq(fragment)
        analyzer = IEP(seq)
        # Calculate charges at pH from 4 to 9
        charges = [analyzer.charge_at_pH(pH) for pH in range(4, 10)]
        charge_matrix.append(charges)

    return charge_matrix  # Returns a 5x6 matrix (5 fragments and 6 pH values)

df_result['charge_matrix'] = df_result['sequence'].apply(calculate_charge_matrix)

#### Udział elementów drugiej struktury peptydu / Contribution of elements of the second peptide structure

In [ ]:
def calculate_secondary_structure_fractions(sequence):
    """
    Calculates the fractions of secondary structure elements (helix, turn, and sheet) for a given protein sequence.

    This function takes a protein sequence as input and uses the ProteinAnalysis module to compute the estimated
    proportions of three secondary structure elements: alpha helix, beta turn, and beta sheet. The result is a
    tuple containing the fraction of each structure type in the sequence.

    Args:
        sequence (str): The amino acid sequence of the protein.

    Returns:
        tuple: A tuple with three values representing the fraction of the sequence predicted to form:
               - Helix (alpha helix)
               - Turn (beta turn)
               - Sheet (beta sheet)
    """
    prot_analysis = ProteinAnalysis(sequence)
    return prot_analysis.secondary_structure_fraction()  # Returns (Helix, Turn, Sheet)

df_result['secondary_structure'] = df_result['sequence'].apply(calculate_secondary_structure_fractions)

#### Hydrofobowość / Hydrophobicity

In [ ]:
def calculate_hydrophobicity(sequence):
    """
    Calculates the hydrophobicity (GRAVY - Grand Average of Hydropathy) of a given protein sequence.

    The GRAVY score indicates the average hydrophobicity or hydrophilicity of the amino acids in the
    sequence. Positive values indicate hydrophobicity, while negative values indicate hydrophilicity.

    Args:
        sequence (str): The amino acid sequence of the protein.

    Returns:
        float: The GRAVY (Grand Average of Hydropathy) value for the sequence, representing the average hydrophobicity.
    """
    prot_analysis = ProtParam.ProteinAnalysis(sequence)
    return prot_analysis.gravy()  # Returns the GRAVY score (average hydrophobicity)

df_result['hydrophobicity'] = df_result['sequence'].apply(calculate_hydrophobicity)

#### Vector of sequence hydrophobicity values ​​for 5 fragments

In [ ]:
def calculate_hydrophobicity_vector(sequence):
    """
    Calculates a hydrophobicity matrix by dividing a protein sequence into 5 fragments and determining the
    hydrophobicity (GRAVY) for each fragment.

    Args:
        sequence (str): The amino acid sequence of the protein.

    Returns:
        list: A list of 5 GRAVY scores, where each score represents the hydrophobicity of one fragment.
    """
    seq_fragments = split_sequence(sequence)  # Split the sequence into 5 fragments
    hydrophobicity_vector = [calculate_hydrophobicity(fragment) for fragment in seq_fragments]
    return hydrophobicity_vector

df_result['hydrophobicity_vector'] = df_result['sequence'].apply(calculate_hydrophobicity_vector)

#### Przewidywanie „Hot-spotów” agregacji / Predicting Aggregation Hot Spots


In [ ]:
def format_fasta(sequence):
    """
    Formats a given DNA or protein sequence into a simplified FASTA format.

    Args:
        sequence (str): The input sequence to be formatted.

    Returns:
        str: The sequence formatted in a simplified FASTA format,
             including a header line ">sp|" followed by the sequence itself.
    """
    formatted_sequence = f">sp|\r\n{sequence}"
    return formatted_sequence

def submit_form_and_get_results(sequence):
    """
    Submits a given sequence to a remote form via a POST request and retrieves the results.

    Args:
        sequence (str): The sequence to be submitted for analysis.

    Returns:
        str or None: The HTML response text from the server if the request is successful (status code 200),
                     or None if there is an error with the request.
    """
    form_url = 'http://bioinf.uab.es/cgi-bin/aap/aap_ov.pl'

    fasta_data = format_fasta(sequence)

    data = {
        'sequence': fasta_data
    }

    response = requests.post(form_url, data=data)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Error sending form. Status code: {response.status_code}")
        return None

def extract_hotspots_values(sequence):
    """
    Extracts numerical hotspot values from the HTML response of a submitted sequence.

    Args:
        sequence (str): The sequence to be analyzed.

    Returns:
        list of float: A list of numerical values representing hotspot analysis results.
                       A warning is printed if fewer than 9 values are found.
                       Returns an empty list if no values are found or if the request fails.
    """
    html_results = submit_form_and_get_results(sequence)
    if html_results:
        soup = BeautifulSoup(html_results, 'html.parser')

        values = []
        td_elements = soup.find_all('td', style="text-align: center; height: 27px;")

        for td in td_elements:
            cell_text = td.get_text(strip=True)
            try:
                value = float(cell_text)
                values.append(value)
            except ValueError:
            # Skip cells that do not contain numerical data
                continue

            # Stop after extracting 9 values (two columns contain identical results)
            if len(values) == 9:
                        break

        if len(values) < 9:
            print("Warning: Less than 9 values found.")

        return values

#df_result['hot_spots_vector'] = df_result['sequence'].apply(extract_hotspots_values)
#sequence = "ATCGATCGATCG"

#results = extract_hotspots_values(sequence)

In [ ]:
def clean_sequence(sequence):
    """
    Removes unnecessary characters from a sequence.
    """
    return sequence.strip()

def submit_sequences(sequences):
    """
    Sends formatted FASTA data to the server.
    """
    form_url = 'http://bioinf.uab.es/cgi-bin/aap/aap_ov.pl'

    fasta_data = "".join([f">sp|sequence_{i + 1}\r\n{clean_sequence(seq)}" for i, seq in enumerate(sequences)])

    data = {
        'sequence': fasta_data
    }

    response = requests.post(form_url, data=data)

    if response.status_code == 200:
        return response.text
    else:
        print(f"Błąd podczas wysyłania danych. Kod statusu: {response.status_code}")
        print("Treść odpowiedzi:", response.text)
        return None

def extract_hotspots_values_sequences(sequences):
    """
    Extracts numerical hotspot values for each sequence from the HTML response of the submitted sequences.

    Args:
        sequences (list of str): List of sequences to be analyzed.

    Returns:
        list of list of float: A nested list where each inner list contains hotspot values
                               for the corresponding input sequence.
    """
    html_results = submit_sequences(sequences)
    if html_results:
        soup = BeautifulSoup(html_results, 'html.parser')

        results = []
        table = soup.find('table')
        if not table:
            print("No table found in the HTML response.")
            return []

        tbody = table.find('tbody')
        if not tbody:
            print("No tbody found in the table.")
            return []

        tr1 = tbody.find('tr')
        if not tr1:
            print("No tr found in the tbody.")
            return []

        tbodys = tr1.find_all('tbody')
        for index, tbody in enumerate(tbodys):
            if not (index == 0 or index == len(tbodys) - 1):  # Ominięcie pierwszego i ostatniego
                list = []
                tds = tbody.find_all('td')
                for td in tds:
                    cell_text = td.get_text(strip=True)
                    try:
                        value = float(cell_text)
                        if len(list) < 9:
                            list.append(value)
                    except ValueError:
                        # Skip cells that do not contain numerical data
                        continue
                if len(list) < 9:
                    print("Warning: Less than 9 values found.")
                results.append(list)


        if len(results) != len(sequences):
            print("Warning: Results and sequences lenghts missmatch.")

        return results
    else:
        return []


# Example usage
sequences = [
    "ATCGATCGATCG",
    "GCTAGCTAGCTA",
    "TTAGGCCTAGGA"
]

results = extract_hotspots_values_sequences(sequences)

#print(results)

#### Zapis do pliku / Saving to file

In [ ]:
#output_file = '/training_data.xlsx'
#df_result.to_excel(output_file, index=False)

#print(df_result)

#### Mapowanie nieznanych aminokwasów / Mapping unknown amino acids

In [ ]:
amino_acid_map = {
    'X': 'A',
    'U': 'C',
    'B': 'D',
    'Z': 'E',
    'O': 'K'
}

valid_amino_acids = set("ACDEFGHIKLMNPQRSTVWY")

def clean_and_map_sequence(sequence):
    """
    Cleans and maps sequences to valid amino acids.

    Args:
        sequence (str): The input protein sequence as a string.

    Returns:
        str: The cleaned and mapped sequence.
    """
    return ''.join([
        amino_acid_map.get(aa, aa) if aa in amino_acid_map or aa in valid_amino_acids else ''
        for aa in sequence
    ])

#### Funkcja zbiorcza z zapisem do pliku / Aggregate function with file output

In [ ]:
def process_sequences(df, output_file):
    """
    Process a DataFrame containing sequences, performing various biochemical calculations,
    and save the results to an output file.

    Args:
        df (pd.DataFrame): A DataFrame containing at least one column 'sequence' with protein sequences.
        output_file (str): The path where the output Excel file should be saved.

    Returns:
        None
    """

    if 'sequence' not in df.columns:
        raise ValueError("Input DataFrame must contain a 'sequence' column.")

    # Step 1: Calculate isoelectric point
    df['isoelectric_point'] = df['sequence'].apply(calculate_isoelectric_point)

    # Step 2: Calculate charge vector for pH range 4-9
    df['charge_vector'] = df['sequence'].apply(calculate_charge_vector)

    # Step 3: Calculate charge matrix for fragments of the sequence
    df['charge_matrix'] = df['sequence'].apply(calculate_charge_matrix)

    # Step 4: Calculate secondary structure fractions (Helix, Turn, Sheet)
    df['secondary_structure'] = df['sequence'].apply(calculate_secondary_structure_fractions)

    # Step 5: Calculate hydrophobicity
    df['hydrophobicity'] = df['sequence'].apply(calculate_hydrophobicity)

    # Step 6: Calculate hydrophobicity vector for sequence fragments
    df['hydrophobicity_vector'] = df['sequence'].apply(calculate_hydrophobicity_vector)

    # Step 7: Extract hotspot values using external form submission
    df['hot_spots_vector'] = df['sequence'].apply(extract_hotspots_values)

    # Step 8: Save the final DataFrame to an Excel file
    df.to_excel(output_file, index=False)
    print(f"Data processing completed and saved to: {output_file}")

### Funkcja zbiorcza dla pojedynczej sekwencji / Aggregate function for single sequence

In [ ]:
def process_sequence_single(sequence):
    """
    Process a protein sequence, performing various biochemical calculations,
    and return a combined feature matrix.

    Args:
        sequence (str): A string containing a protein sequence.

    Returns:
        np.ndarray: A NumPy array containing combined feature matrix.
    """

    if not isinstance(sequence, str):
        raise ValueError("Input must be a string containing a protein sequence.")

    sequence = clean_and_map_sequence(sequence)

    # Step 0: Create DataFrame from the sequence
    df = pd.DataFrame({'sequence': [sequence]})

    # Step 1: Calculate isoelectric point
    df['isoelectric_point'] = df['sequence'].apply(calculate_isoelectric_point)

    # Step 2: Calculate charge vector for pH range 4-9
    df['charge_vector'] = df['sequence'].apply(calculate_charge_vector)

    # Step 3: Calculate charge matrix for fragments of the sequence
    df['charge_matrix'] = df['sequence'].apply(calculate_charge_matrix)

    # Step 4: Calculate secondary structure fractions (Helix, Turn, Sheet)
    df['secondary_structure'] = df['sequence'].apply(calculate_secondary_structure_fractions)

    # Step 5: Calculate hydrophobicity
    df['hydrophobicity'] = df['sequence'].apply(calculate_hydrophobicity)

    # Step 6: Calculate hydrophobicity vector for sequence fragments
    df['hydrophobicity_vector'] = df['sequence'].apply(calculate_hydrophobicity_vector)

    # Step 7: Extract hotspot values using external form submission
    df['hot_spots_vector'] = df['sequence'].apply(extract_hotspots_values)

    # Step 8: Convert columns to numpy arrays
    isoelectric_point = df['isoelectric_point'].to_numpy().reshape(-1, 1)
    hydrophobicity = df['hydrophobicity'].to_numpy().reshape(-1, 1)

    def parse_vector(x):
        if isinstance(x, str):
            return np.fromstring(x.strip('array([])').replace('\n', ''), sep=', ')
        return np.array(x)

    df['charge_vector'] = df['charge_vector'].apply(parse_vector)
    df['secondary_structure'] = df['secondary_structure'].apply(parse_vector)
    df['hot_spots_vector'] = df['hot_spots_vector'].apply(parse_vector)

    charge_vector = np.array([np.array(x) for x in df['charge_vector']])
    secondary_structure = np.array([np.array(x) for x in df['secondary_structure']])
    hot_spots_vector = np.array([np.array(x) for x in df['hot_spots_vector']])

    # Step 9: Combine features into a single array
    combined_features = np.concatenate((
        isoelectric_point,
        hydrophobicity,
        charge_vector,
        secondary_structure,
        hot_spots_vector
    ), axis=1)

    combined_features = combined_features.astype(np.float32)

    return combined_features

combined_features = process_sequence_single("RHVHLRARASGSARIYQAGRDQHITER")
print(combined_features)

[[ 1.1705283e+01 -1.1407408e+00  8.4973450e+00  7.0674934e+00
   5.4767504e+00  4.0244966e+00  3.2585604e+00  2.9366555e+00
   2.2222222e-01  1.8518518e-01  2.2222222e-01 -3.8600001e-01
   0.0000000e+00  0.0000000e+00  9.4099998e-01  0.0000000e+00
  -8.8170004e+00  3.5000000e-02  0.0000000e+00 -3.6599998e+01]]


### Funkcja zbiorcza dla listy sekwencji / Aggregate function for list of sequences

In [ ]:
def process_sequences(sequences):
    """
    Process a list of protein sequences, performing various biochemical calculations,
    and return a list of combined feature matrices.

    Args:
        sequences (list): A list of strings, each containing a protein sequence.

    Returns:
        list: A list of NumPy arrays, each containing a combined feature matrix for a sequence.
    """
    if not isinstance(sequences, list) or not all(isinstance(seq, str) for seq in sequences):
        raise ValueError("Input must be a list of strings containing protein sequences.")

    combined_feature_matrices = []
    hotspots = extract_hotspots_values_sequences(sequences)
    if len(hotspots) != len(sequences):
        print("Warning: Results and sequences lenghts missmatch.")
        return []
    for idx, sequence in enumerate(sequences):
        sequence = clean_and_map_sequence(sequence)
        # Step 0: Create DataFrame from the sequence
        df = pd.DataFrame({'sequence': [sequence]})

        # Step 1: Calculate isoelectric point
        df['isoelectric_point'] = df['sequence'].apply(calculate_isoelectric_point)

        # Step 2: Calculate charge vector for pH range 4-9
        df['charge_vector'] = df['sequence'].apply(calculate_charge_vector)

        # Step 3: Calculate charge matrix for fragments of the sequence
        df['charge_matrix'] = df['sequence'].apply(calculate_charge_matrix)

        # Step 4: Calculate secondary structure fractions (Helix, Turn, Sheet)
        df['secondary_structure'] = df['sequence'].apply(calculate_secondary_structure_fractions)

        # Step 5: Calculate hydrophobicity
        df['hydrophobicity'] = df['sequence'].apply(calculate_hydrophobicity)

        # Step 6: Calculate hydrophobicity vector for sequence fragments
        df['hydrophobicity_vector'] = df['sequence'].apply(calculate_hydrophobicity_vector)

        # Step 7: Extract hotspot values using external form submission

        # Step 8: Convert columns to numpy arrays
        isoelectric_point = df['isoelectric_point'].to_numpy().reshape(-1, 1)
        hydrophobicity = df['hydrophobicity'].to_numpy().reshape(-1, 1)

        def parse_vector(x):
            if isinstance(x, str):
                return np.fromstring(x.strip('array([])').replace('\n', ''), sep=', ')
            return np.array(x)

        df['charge_vector'] = df['charge_vector'].apply(parse_vector)
        df['secondary_structure'] = df['secondary_structure'].apply(parse_vector)

        charge_vector = np.array([np.array(x) for x in df['charge_vector']])
        secondary_structure = np.array([np.array(x) for x in df['secondary_structure']])
        hot_spots_vector = [np.array(hotspots[idx])]

        # Step 9: Combine features into a single array
        combined_features = np.concatenate((
            isoelectric_point,
            hydrophobicity,
            charge_vector,
            secondary_structure,
            hot_spots_vector
        ), axis=1)

        combined_features = combined_features.astype(np.float32)
        combined_feature_matrices.append(combined_features)

    return np.array(combined_feature_matrices)

#print(process_sequences(["RHVHLRARASGSARIYQAGRDQHITER", "RHVHLRARASGSARIYQAGRDQHITER"]))